### Generate Dependency Graph for Numpy, Max Depth = 4

In [2]:
from pygraas import DependencyGraph, VulnerabilityGraph
g = DependencyGraph(package_name="numpy", package_url="https://github.com/numpy/numpy")
graph = g.build_graph(max_bacon=4)

Cloning into 'numpy'...
dummymodule.py:140: WARNING: SKIPPING ILLEGAL MODULE_NAME: numpy.numpy._pyinstaller.hook-numpy
dummymodule.py:140: WARNING: SKIPPING ILLEGAL MODULE_NAME: numpy.numpy._pyinstaller.tests.pyinstaller-smoke



changing __main__ => _dummy_numpy.py
changing __main__ => _dummy_numpy.py
changing __main__ => _dummy_numpy.py
changing __main__ => _dummy_numpy.py
Graph built with 292 nodes and 1494 edges.


### Generate Vulnerability Graph based on the Dependency Graph

Vulnerability Graph marks the external packages which are Vulnerable, also add's the CVEs, advisory and 
vulnerable package versions as node attributes.

In [3]:
v = VulnerabilityGraph(g)
v.build_vulnerability_graph()

#### List external vulnerable packages based on the Safety-DB Database

In [4]:
vulnerables = v.get_vulnerables()
print(vulnerables)

['numpy', 'psutil', 'setuptools']


### Read Details

In [5]:
v.get_vulnerables(details=True)[0]["numpy"]["version"]

['<1.13.2',
 '<1.16.3',
 '<1.21.0rc1',
 '<1.22.0',
 '<1.22.0',
 '<1.22.2',
 '<1.8.1',
 '<1.8.1']

In [6]:
v.get_vulnerables(details=True)[0]["numpy"]["CVE"]

['CVE-2017-12852',
 'CVE-2019-6446',
 'CVE-2021-33430',
 'CVE-2021-34141',
 'CVE-2021-41496',
 'CVE-2021-41495',
 'CVE-2014-1858',
 'CVE-2014-1859']

In [7]:
v.get_vulnerables(details=True)[0]["numpy"]["advisory"]

['Numpy 1.13.2 includes a fix for CVE-2017-12852: The numpy.pad function in Numpy 1.13.1 and older versions is missing input validation. An empty list or ndarray will stick into an infinite loop, which can allow attackers to cause a DoS attack.\r\nhttps://github.com/numpy/numpy/issues/9560#issuecomment-322395292',
 'Numpy 1.16.3 includes a fix for CVE-2019-6446: It uses the pickle Python module unsafely, which allows remote attackers to execute arbitrary code via a crafted serialized object, as demonstrated by a numpy.load call.\r\nNOTE: Third parties dispute this issue because it is  a behavior that might have legitimate applications in (for example) loading serialized Python object arrays from trusted and authenticated  sources.\r\nhttps://github.com/numpy/numpy/commit/89b688732b37616c9d26623f81aaee1703c30ffb',
 'Numpy 1.21.0rc1 includes a fix for CVE-2021-33430: A Buffer Overflow vulnerability in the PyArray_NewFromDescr_int function of ctors.c when specifying arrays of large dimens

`np.pad` seems vulnerable, we can restrict it using `pygraas.RestrictionLayer` .

Here, just for demonstration purposes, we'll restricted the usage of `numpy.zeros` which is called internally in `skimage.filters.sobel` .

In [16]:
from pygraas import RestrictionLayer
from skimage.filters import sobel
import numpy as np

l = RestrictionLayer(["numpy.zeros"])
l.activate()

image = np.random.random((4000, 4000))
sobel(image)

RestrictedFunctionError: The function 'numpy.zeros' is blocked.

#### Degree of the "psutil" node 

In [9]:
v.get_degree(vulnerables[1])

{'in_degree': 10, 'out_degree': 1}

#### List of Public Nodes that have a path to vulnerable "setuptools"

In [10]:
# Show only the first 20 entries for demonstration.
v.get_vulnerable_public_nodes(vulnerables[2])[0: 20]

[{'bz2 - setuptools': True},
 {'dis - setuptools': True},
 {'heapq - setuptools': True},
 {'dataclasses - setuptools': True},
 {'collections - setuptools': True},
 {'lzma - setuptools': True},
 {'codecs - setuptools': True},
 {'errno - setuptools': True},
 {'fcntl - setuptools': True},
 {'math - setuptools': True},
 {'enum - setuptools': True},
 {'io - setuptools': True},
 {'email - setuptools': True},
 {'importlib - setuptools': True},
 {'difflib - setuptools': True},
 {'getopt - setuptools': True},
 {'copy - setuptools': True},
 {'base64 - setuptools': True},
 {'doctest - setuptools': True},
 {'argparse - setuptools': True}]